In [ ]:
!pip install sqlalchemy

You are using pip version 8.1.1, however version 10.0.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [22]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from IPython.display import display
import sys
import re
import numpy as np
from difflib import SequenceMatcher
from sklearn.neighbors import KNeighborsClassifier

The first few cells are focused on data cleaning and oranizing the data so it run through the classifier algorithms. The get_population_data method reads in the precint demographics.

In [39]:
def get_population_data():
    with open('PrecinctStat.csv') as gfile:
        x=0
        data = list()
        for lno, line in enumerate(gfile, start=0):
            if x==0:
                key=line.split(',')
            else:
                dat=line.split(',')
                data.append(dat)
            x=1
    return data


Generates a list of keys to indicate which features we would be focusing on

In [40]:
def get_key():
    with open('PrecinctStat.csv') as gfile:
        x=0
        key = list()
        for lno, line in enumerate(gfile, start=0):
            if x==0:
                key=line.split(',')
            x=1
    return key

In [41]:
key = get_key()
x=0;
for i in key:
    #print x , "   ", i
    x=x+1
4,5,6,45,46,47,48,49,50 

(4, 5, 6, 45, 46, 47, 48, 49, 50)

The county and precint labels in the demographic data and the election results data differed slightly in some cases so we wrote a similarity function to match the data together

In [42]:
def similar(a,b):
    max =0 ;
    temp = .01*.1
    tempIndex =0
    for x in range(len(b)):
        temp = SequenceMatcher(None, a,b[x]).ratio()  
        if temp > max:
            max=temp
            tempIndex= x    
    return tempIndex, max
     

This function matches the demographic data from each precinct to their respective election results.

In [43]:
 def pair_data(data, precincts, demVotes, repVotes, totalVotes, winner):
    election_results = list()
    pElection = list()
    pCensus = list()
    for x in range(len(data)):
        pC = (data[x][2]+data[x][1]).lower()
        pC = re.sub('\W+','', pC)
        pCensus.append(pC)
        
        pE = (precincts[x][0] + precincts[x][1]).lower()
        pE = re.sub('\W+','', pE)
        pElection.append(pE)
    count =0;
    voter_info=list()
    delIndex = list()
    delIndexp = list()
    for x in range(len(pCensus)):
        vInfo=list()
        '''if x%100 == 0:
            print x'''
        if pCensus[x] in pElection:
            i = pElection.index(pCensus[x])
            vInfo.append(precincts[i][1]+precincts[i][0])
            vInfo.append(demVotes[i]) 
            vInfo.append(repVotes[i])
            vInfo.append(totalVotes[i])
            vInfo.append(winner[i])
            voter_info.append(vInfo)
            #print precincts[i][0]+precincts[i][1], "    ", data[x][2]+data[x][1], "    ", 1

        else:
            [i,b] = similar(pCensus[x], pElection)
            if(b>.93):
                vInfo.append(precincts[i][1]+precincts[i][0])
                vInfo.append(demVotes[i]) 
                vInfo.append(repVotes[i])
                vInfo.append(totalVotes[i])
                vInfo.append(winner[i])
                voter_info.append(vInfo)
                #print precincts[i][0]+precincts[i][1], "    ", data[x][2]+data[x][1], "    ", b
            else:
                delIndex.append(x)
        
    for x in sorted(delIndex, reverse=True):
        del data[x]

    return data, voter_info    

Code to read in the election outcomes on a precint level for the 2004 and 2006 elections

In [44]:
def read_csv_2004_06(elec_year):
    #select the election year that you wish to read 
    #elec_year = "ElecitonResults_2004.csv"
    #elec_year = "ElectionResults_2006.csv"
    
    
    with open(elec_year, 'r') as f:
        precincts = list()
        contest = list()
        candidate = list()
        party = list()
        demVotes = list()
        repVotes = list()
        totalVotes = list()
        winner = list()
        for line in f: 
            fields = line.split(',')
            contest_type = fields[4]
            pc = (fields[3].strip(), fields[0].strip())
            #We only want the fields that have to do with US HOUSE of Rep elections
            if "US CONGRESS" in contest_type and len(fields)==9: 
                
                #if the precinct hasn't been added to the list yet add it in with the relevant fields 
                if pc not in precincts: 
                    precincts.append(pc)
                    contest.append(contest_type)
                    #candidate.append(fields[5])
                    #party.append(fields[6])
                    if fields[6] == "DEM": 
                        demVotes.append(int(fields[7]))
                        repVotes.append(-1)
                    elif fields[6] == "REP":
                        repVotes.append(int(fields[7]))
                        demVotes.append(-1)
                    else: 
                        repVotes.append(-1)
                        demVotes.append(-1)
                        
                    totalVotes.append(int(fields[7]))
                    winner.append(fields[6])
                    
                #if the precinct has already been added, update the fields accordingly
                else: 
                    index = precincts.index(pc)
                    if fields[6] == "DEM": 
                        demVotes[index] = int(fields[7])
                        curVotes = totalVotes[index]
                        newTotal = curVotes + int(fields[7])
                        totalVotes[index] = newTotal
                    elif fields[6] == "REP": 
                        repVotes[index] = int(fields[7])
                        curVotes = totalVotes[index]
                        newTotal = curVotes + int(fields[7])
                        totalVotes[index] = newTotal
                    else: 
                        curVotes = totalVotes[index]
                        newTotal = curVotes + int(fields[7])
                        totalVotes[index] = newTotal
                    
                    if demVotes[index] > repVotes[index]: 
                        winner[index] = "DEM"
                    else: 
                        winner[index] = "REP"
                
    #print "Precinct: ", precincts[1], " Contest: ", contest[1], " Dem Votes: ", demVotes[1], " Rep Votes: ", repVotes[1], " Total Votes: ", totalVotes[1], " Winner: ", winner[1]
        
    data = get_population_data()
    return pair_data(data, precincts, demVotes, repVotes, totalVotes, winner)
        

Reading in the data and storing the features in outcomes in x and y for 2004 and 2006 elections

In [45]:
[x1,y1]=read_csv_2004_06("ElecitonResults_2004.csv")
[x2,y2]=read_csv_2004_06("ElectionResults_2006.csv")


Reading in the election outcomes for the 2008, 2010, and 2012 elections. The format of these csv files were different than the 2004 and 2006 elections so the code differed slightly. We would also like to note we did not use the data from the 2010 and 2012 elections because they were missing data that caused erros when pairing this data with the precinct demographic data.

In [47]:
#read in CSV file and store relevant components --> for 2008, 2010, and 2012
    
def read_csv_2008_10_12(elec_year):
    
    
    
    with open(elec_year, 'r') as f:
        precincts = list()
        contest = list()
        candidate = list()
        party = list()
        demVotes = list()
        repVotes = list()
        totalVotes = list()
        winner = list()
        for line in f: 
            fields = line.split(',')
            contest_type = fields[5]
            pc = (fields[1].strip(), fields[0].strip())
            #We only want the fields that have to do with US HOUSE of Rep elections
            if "US HOUSE OF REPRESENTATIVES" in contest_type and len(fields)==14: 
                #print "test"
                if pc not in precincts: 
                    
                    precincts.append(pc)
                    contest.append(contest_type)
                    #candidate.append(fields[5])
                    #party.append(fields[6])
                    if fields[8] == "DEM": 
                        demVotes.append(int(fields[12]))
                        repVotes.append(-1)
                    elif fields[8] == "REP":
                        repVotes.append(int(fields[12]))
                        demVotes.append(-1)
                    else: 
                        repVotes.append(-1)
                        demVotes.append(-1)
                        
                    totalVotes.append(int(fields[12]))
                    winner.append(fields[8])
                    
                #if the precinct has already been added, update the fields accordingly
                else: 
                    index = precincts.index(pc)
                    if fields[8] == "DEM": 
                        demVotes[index] = int(fields[12])
                        curVotes = totalVotes[index]
                        newTotal = curVotes + int(fields[12])
                        totalVotes[index] = newTotal
                    elif fields[8] == "REP": 
                        repVotes[index] = int(fields[12])
                        curVotes = totalVotes[index]
                        newTotal = curVotes + int(fields[12])
                        totalVotes[index] = newTotal
                    else: 
                        curVotes = totalVotes[index]
                        newTotal = curVotes + int(fields[12])
                        totalVotes[index] = newTotal
                    
                    if demVotes[index] > repVotes[index]: 
                        winner[index] = "DEM"
                    else: 
                        winner[index] = "REP"
                
                """#print contest_type 
                precincts.append(fields[1].strip())
                contest.append(contest_type)
                candidate.append(fields[6])
                party.append(fields[8])
                voteCount.append(fields[12])"""
                
    #print "Precinct: ", precincts[0], " Contest: ", contest[0], " Dem Votes: ", demVotes[0], " Rep Votes: ", repVotes[0], " Total Votes: ", totalVotes[0], " Winner: ", winner[0]

    data = get_population_data()
    return pair_data(data, precincts, demVotes, repVotes, totalVotes, winner)

Reading the election results for 2008 and storing them in x and y

In [48]:
[x3,y3] = read_csv_2008_10_12("ElectionResults_2008.csv")

#We were going to use the 2010 and 2012 elections but they had missing data which caused errors when cleaning the data
#[x4,y4] = read_csv_2008_10_12("ElectionResults_2010.csv")
#[x5,y5] = read_csv_2008_10_12("ElectionResults_2012.csv")

This our algorithm for predicting election results using a nearest neighbor classifer.

In [77]:
from sklearn import cross_validation, naive_bayes

def run_classifier(feat, out):
    k=9
    features = [4,5,6,45,46,47,48,49,50] 
    print"Running K nearest neighbors with k=", k , "on Congressional year 2008 NC with features as";
    entries = list()
    outcomes = list()
    """for x in features:
        print key[x]"""
    for x in range(len(out)):
        temp = list()
        for y in features:
            temp.append(feat[x][y])  
        entries.append(temp)
    for y in range(len(out)):
        outcomes.append(out[y][4])

    test_entries = list()
    """for x in features:
        print key[x]"""

    #print len(entries), len(entries[0])
    #print "with ",len(outcomes), " entries"
    neigh = KNeighborsClassifier(n_neighbors = k)




    X_train, X_test, y_train, y_test \
        = cross_validation.train_test_split(entries, outcomes, test_size=0.33, random_state=216)
    neigh.fit(X_train, y_train)
    print 'train accuracy: {:.4f}'.format(neigh.score(X_train, y_train))
    print 'test accuracy: {:.4f}'.format(neigh.score(X_test, y_test))
    nf = 10
    scores = cross_validation.cross_val_score(neigh, entries, outcomes, cv=10)
    print scores
    print 'accuracy: {:.4f} (+/- {:.4f})'.format(scores.mean(), scores.std()*2)

    neigh.fit(entries, outcomes)
    y = neigh.predict(entries)

    correct =0;
    #print len(y)
    for i in range(len(y)):
        #print y[i], "   ", y_test[i]
        if y[i] == outcomes[i]:
            correct += 1
    print "Predicted ", correct, "results out of ", len(y), "giving an accuracy of ", float(correct)/float(len(y))



In [78]:
run_classifier(x1,y1)

Running K nearest neighbors with k= 9 on Congressional year 2008 NC with features as
train accuracy: 0.8405
test accuracy: 0.7887
[0.82485876 0.72881356 0.81920904 0.84745763 0.78531073 0.75
 0.85714286 0.78857143 0.79428571 0.80571429]
accuracy: 0.8001 (+/- 0.0761)
Predicted  1471 results out of  1761 giving an accuracy of  0.835320840432


In [79]:
run_classifier(x2,y2)

Running K nearest neighbors with k= 9 on Congressional year 2008 NC with features as
train accuracy: 0.7923
test accuracy: 0.7304
[0.76646707 0.61676647 0.86227545 0.72289157 0.79518072 0.59036145
 0.71686747 0.6686747  0.77575758 0.80606061]
accuracy: 0.7321 (+/- 0.1640)
Predicted  1310 results out of  1661 giving an accuracy of  0.788681517158


In [80]:
run_classifier(x3,y3)

Running K nearest neighbors with k= 9 on Congressional year 2008 NC with features as
train accuracy: 0.7761
test accuracy: 0.7271
[0.77857143 0.63309353 0.79856115 0.67625899 0.67625899 0.64028777
 0.57971014 0.77536232 0.74637681 0.80434783]
accuracy: 0.7109 (+/- 0.1511)
Predicted  1076 results out of  1387 giving an accuracy of  0.775775054074


This is our algorithm for predicting election results using Gaussian Naive Bayes

In [83]:
from sklearn import cross_validation, naive_bayes
def run_bayes(feat, out):    
    
    features = [4,5,6,45,46,47,48,49,50] 
    print"Running naive bayes on Congressional year 2008 NC with features as";
    entries = list()
    outcomes = list()
    """for x in features:
        print key[x]"""
    for x in range(len(out)):
        temp = list()
        for y in features:
            temp.append(int(feat[x][y]))  
        entries.append(temp)
    for y in range(len(out)):
        outcomes.append(out[y][4])

    test_entries = list()
    """for x in features:
        print key[x]"""

    #print len(entries), len(entries[0])
    #print "with ",len(outcomes), " entries"


    classifier = naive_bayes.GaussianNB()
    X_train, X_test, y_train, y_test \
        = cross_validation.train_test_split(entries, outcomes, test_size=0.33, random_state=216)
    classifier.fit(X_train, y_train)
    
    print 'train accuracy: {:.4f}'.format(classifier.score(X_train, y_train))
    print 'test accuracy: {:.4f}'.format(classifier.score(X_test, y_test))
    
    classifier.fit(entries, outcomes)
    y = classifier.predict(entries)

    correct =0;
    #print len(y)
    for i in range(len(y)):
        #print y[i], "   ", y_test[i]
        if y[i] == outcomes[i]:
            correct += 1
    print "Predicted ", correct, "results out of ", len(y), "giving an accuracy of ", float(correct)/float(len(y))






In [84]:
run_bayes(x1,y1)

Running naive bayes on Congressional year 2008 NC with features as
train accuracy: 0.7379
test accuracy: 0.7302
Predicted  1273 results out of  1761 giving an accuracy of  0.722884724588


In [85]:
run_bayes(x2,y2)

Running naive bayes on Congressional year 2008 NC with features as
train accuracy: 0.6178
test accuracy: 0.6357
Predicted  1033 results out of  1661 giving an accuracy of  0.621914509332


In [86]:
run_bayes(x3,y3)

Running naive bayes on Congressional year 2008 NC with features as
train accuracy: 0.6308
test accuracy: 0.6441
Predicted  874 results out of  1387 giving an accuracy of  0.630136986301
